In [2]:
# Download file. 
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/all_peaks.csv
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/cicero_connections.csv
    

/root/anaconda3/envs/celloracle/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/root/anaconda3/envs/celloracle/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/root/anaconda3/en

,Peak1,Peak2,coaccess
1,chr10_100006139_100006389,chr10_99774288_99774570,-0.003546
2,chr10_100006139_100006389,chr10_99825945_99826237,-0.027536
3,chr10_100006139_100006389,chr10_99830012_99830311,0.009588
4,chr10_100006139_100006389,chr10_99833211_99833540,-0.008067
5,chr10_100006139_100006389,chr10_99941805_99941955,0.000000


In [ ]:
# Load scATAC-seq peak list.
import pandas as pd
from celloracle import motif_analysis as ma
import celloracle as co

peaks = pd.read_csv("all_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv("cicero_connections.csv", index_col=0)
cicero_connections.head()

In [4]:
##!! Please make sure to specify the correct reference genome here
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="mm10") 

# Check results
tss_annotated.tail()

que bed peaks: 86935
tss peaks in que: 17238


,chr,start,end,gene_short_name,strand
17233,chr1,55130650,55132118,Mob4,+
17234,chr6,94499875,94500767,Slc25a26,+
17235,chr19,45659222,45660823,Fbxw4,-
17236,chr12,100898848,100899597,Gpr68,-
17237,chr4,129491262,129492047,Fam229a,-


In [5]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated, 
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(44309, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100006139_100006389,Tmtc3,0.017915
1,chr10_100015291_100017830,Kitl,1.000000
2,chr10_100018677_100020384,Kitl,0.146517
3,chr10_100050858_100051762,Kitl,0.069751
4,chr10_100052829_100053395,Kitl,0.202670


In [6]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

## Test 1: if there any in the coaccess resulted from cicero

In [10]:
print('number of 1: ', cicero_connections[cicero_connections.coaccess.abs()==1])

number of 1:  Empty DataFrame
Columns: [Peak1, Peak2, coaccess]
Index: []


## Test 2: what portion of shortlisted peak-gene is cicero-based

In [18]:
integrated_filter = integrated[integrated.coaccess >= 0.8]
print('all peaks in cicero results that have more than 0.8 score: ', cicero_connections[cicero_connections.coaccess>=0.8].shape[0])
print('number of peak-gene that come from cicero: ', integrated_filter[integrated_filter.coaccess<1].shape[0])

all peaks in cicero results that have more than 0.8 score:  56
number of peak-gene that come from cicero:  0
